# File structure

There are three main parts:
descrpiption
price
tax

# Import file
the import file reads as valid json
camping_sites_2.json

# Structure
within python it is considered as two main data structures:
dictionaries -> key value pairs
lists -> array of list items, in the data structure each list item is a dictionary

# Goal
flattening the data structure means constructing a csv table
each dictionary key represents a column in that table

since the data structure is nested each column heading should should also show all parent keys separated by a period
partent.child

whitespaces should be replaced by an _
ex Zeltplatz_parzelliert not Zeltplatz parzelliert

each element in the data structure should be represented by a row in the table

example data:
"description.@context", "description.@type":
"http://schema.org","Campground"

# Features
There is one exception how the csv table should be built:
    "description.amenityFeature": [
    {
        "@type": "LocationFeatureSpecification",
        "name": "Ausguss f&uuml;r chemische Toiletten"
    },
    {
        "@type": "LocationFeatureSpecification",
        "name": "Baden in nat&uuml;rlichem Gew&auml;sser"
    }

In this case the @type should be ignored. Instead the columns should look like this:

"description.amenityFeature.Ausguss_für_chemische_Toiletten"
"True"

If the amenity is not present in the list the value should be set to "false"

# Problems
- For some reason the char encoding is not correct
- Not every campsite has the same elements. The columns are variable. The price elements especially.
- In the beginning not all amenities are known. the very last step of making the list should be to set the boolean of the amenity to false, if it was not listed previously.

In [1]:
import html
import json
from collections.abc import MutableMapping

# Function to canonize a key. Spaces are replaced by "_", @ is removed,
# HTML entities such as &uuml; are decoded.
def canonize_key(key):
    key = html.unescape(key)
    key = key.replace(' ', '_').replace('@', '')
    return key
    
    
# Coder for flattening a dict in python. Copied from: 
# https://www.freecodecamp.org/news/how-to-flatten-a-dictionary-in-python-in-4-different-ways/

def _flatten_dict_gen(d, parent_key, sep):
    for key, value in d.items():
        new_key = canonize_key(parent_key + sep + key if parent_key else key)
        if isinstance(value, MutableMapping):
            yield from flatten_dict(value, new_key, sep=sep).items()
        else:
            yield new_key, value

def flatten_dict(d: MutableMapping, parent_key: str = '', sep: str = '.'):
    return dict(_flatten_dict_gen(d, parent_key, sep))


# Function to flatten amenities to "AmenityName":true
def flatten_amenities(site):
    all_amenities = []
    feature_key = 'description.amenityFeature'
    if feature_key in site:
        for amenity in site[feature_key]:
            amenity_key = feature_key + '.' + canonize_key(amenity['name'])
            site[amenity_key] = True
            all_amenities.append(amenity['name'])
        del site[feature_key]
        all_amenities.sort()
        site['description.amenities'] = all_amenities
    return site

# Read camping_sites.json and flatten it

try:
    f = open('camping_sites.json')
    sites = json.load(f) # returns JSON object as a dictionary
    f.close()
except Exception as e:
    sites = []
    print('Error:', e)

flatsites = []
for site in sites:
    flatsite = flatten_dict(sites[site])
    flatsite = flatten_amenities(flatsite)
    flatsites.append(flatsite)

# print(json.dumps(flatsites[1:2], indent=4, ensure_ascii=False))
  


Now we find all unique amenityFeatures, and set all missing features to False in all entries

In [2]:
# Find all amenityFeature types
amenityFeatures = {}
for flatsite in flatsites:
    for key in flatsite:
        parts = key.split('.')
        if parts[0] == 'description' and parts[1] == 'amenityFeature':
            amenityFeatures[key] = True

print("Number of distinct amenityFeatures:", len(amenityFeatures))

# Add all missing features in all sites and set them to False
for flatsite in flatsites:
    for feature in amenityFeatures:
        if not feature in flatsite:
            flatsite[feature] = False

Number of distinct amenityFeatures: 109


# Data enrichement

In [3]:
# Add number of features to all campsites
for flatsite in flatsites:
    count = 0
    for key in flatsite:
        if 'description.amenityFeature' in key:
            if flatsite[key] == True:
                count += 1
    flatsite['description.amenityFeature.total'] = count
    # print("count", count)

In [4]:
# Add price median, min, max and amount of price categories

import statistics as st

for flatsite in flatsites:
    prices = [] 
    for key in flatsite:
        if 'price.' in key: # all keys from the group price
            if '.price' in key: # only the actual prices, not the category names
                prices.append(float(flatsite[key]))
    flatsite['price.categories'] = len(prices)
    flatsite['price.max'] = max(prices)
    flatsite['price.min'] = min(prices)
    flatsite['price.median'] = st.median(prices)

For the evaluation of the language regions and the cantons the required data is added to the camping sites in the description area.

Data is provided by the swiss post. CC-BY licence.
Downloaded at: https://swisspost.opendatasoft.com/explore/dataset/plz_verzeichnis_v2/information/?disjunctive.postleitzahl

In [5]:
import re
import pandas as pd

# opening the downloaded file with the data 
plz_df = pd.read_csv('plz_verzeichnis_v2.csv', sep = ";")
# dropping plz duplicates, because duplicates do not change the outcome of the evaluation
plz_df = plz_df.drop_duplicates(subset='POSTLEITZAHL', keep="first") 

# function to retrieve row from plz data with data about one single plz
def get_plz_data(plz):
    return plz_df.loc[lambda df: plz_df['POSTLEITZAHL'] == plz]
 
# function for retrieving language region
def get_lang(plz_data_df):
    lang_dict = {
        '1':'German', 
        '2':'French', 
        '3':'Italian'
    }
    # print(plz_data_df)
    lang = plz_data_df.iloc[0]['SPRACHCODE']
    lang = lang_dict[str(lang)]
    return lang
    

# list of english canton names downloaded from: https://en.wikipedia.org/wiki/Cantons_of_Switzerland
cantons_df = pd.read_csv('cantons_list.csv', index_col = 'ABBR', comment='#')

# function for retrieving language region
def get_canton(plz_data_df):
    canton = plz_data_df.iloc[0]['KANTON']
    canton = cantons_df.loc[canton]['EN']
    return canton

# correcting wrong plz in the data, any other wrong plz can be corrected here
def correcting_plz(plz):
    if plz == 1539:
        plz = 1530
    return plz

In [6]:
# adding plz, language area and canton to the data
for flatsite in flatsites:
    plz = re.findall(r'\d\d\d\d', flatsite['description.address.streetAddress'])
    plz = int(plz[0])
    plz = correcting_plz(plz)
    # print('here is the plz', plz)
    plz_data = get_plz_data(plz)
    flatsite['description.plz'] = plz
    flatsite['description.langArea'] = get_lang(plz_data)
    flatsite['description.canton'] = get_canton(plz_data)

In [7]:
for item in flatsites:
    for key, value in item.items():
        if key == 'tax.main_season.Preis_Erwachsene' and value == 1214:
            print('old values:', key, value)
            index = flatsites.index(item)
            flatsites[index][key] = 14
            print('new value:', flatsites[index][key])

old values: tax.main_season.Preis_Erwachsene 1214.0
new value: 14


# Save data to files

In [8]:
# Write sites to camping_sites_flat.json
json_object = json.dumps(flatsites, indent=4, sort_keys=True, ensure_ascii=False)
with open("camping_sites_flat.json", "w") as outfile:
    outfile.write(json_object)

In [9]:
import pandas as pd
import json

    
df = pd.read_json('camping_sites_flat.json')
df.head(10)


,description.address.addressCountry,description.address.streetAddress,description.address.type,description.aggregateRating.ratingValue,description.aggregateRating.reviewCount,description.aggregateRating.type,description.amenities,description.amenityFeature.ACS_Camping_Card,description.amenityFeature.Angeln,description.amenityFeature.Animationsprogramm,...,price.Standplatz_VW-Camper_(bis_2_Personen).description,price.Standplatz_VW-Camper_(bis_2_Personen).price,price.Standplatz_Wohnmobil_/_Wohnwagen_(bis_4_Personen).description,price.Standplatz_Wohnmobil_/_Wohnwagen_(bis_4_Personen).price,price.stall-liechti.description,price.stall-liechti.price,price.2-Personen-Zelt.description,price.2-Personen-Zelt.price,price.Stellplatz.description,price.Stellplatz.price
0,Switzerland,"Wilbrunnenstrasse 81, 6314 Unterägeri",PostalAddress,4.4,444.0,AggregateRating,"[Angeln, Aufenthaltsraum, Ausguss für chemisch...",False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Switzerland,"Hofmattstrasse 40, 4663 Aarburg (AG)",PostalAddress,4.0,290.0,AggregateRating,"[Angeln, Aufenthaltsraum, Ausguss für chemisch...",False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Switzerland,"Zona Acquacalda 1, 6718 Blenio",PostalAddress,4.2,18.0,AggregateRating,"[Biken, Duschen mit Warmwasser, Einzelduschkab...",False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Switzerland,"Via Lucomagno 163, 6716 Acquarossa",PostalAddress,4.6,62.0,AggregateRating,"[Angeln, Aufenthaltsraum, Ausguss für chemisch...",False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Switzerland,"Via Molinazzo 9, 6982 Agno",PostalAddress,4.4,853.0,AggregateRating,"[Angeln, Aufenthaltsraum, Ausguss für chemisch...",False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Switzerland,"Via Molinazzo 21, 6982 Agno",PostalAddress,NaN,NaN,NaN,"[Angeln, Ausguss für chemische Toiletten, Bade...",False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Switzerland,"Via Acquacalda 15, 6982 Agno",PostalAddress,4.1,120.0,AggregateRating,"[Angeln, Aufenthaltsraum, Ausguss für chemisch...",False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Switzerland,"Niederstad 6, 6053 Alpnach",PostalAddress,4.3,189.0,AggregateRating,"[Angeln, Ausguss für chemische Toiletten, Bade...",False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Switzerland,"Flüelerstrasse 112, 6460 Altdorf",PostalAddress,4.2,288.0,AggregateRating,"[Aufenthaltsraum, Ausguss für chemische Toilet...",False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Switzerland,"Mennstrasse 2, 9423 Altenrhein",PostalAddress,4.5,367.0,AggregateRating,"[Angeln, Ausguss für chemische Toiletten, Baby...",False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df.shape

(403, 793)